In [2]:
from IPython.display import HTML
from IPython.display import display

tag = HTML('''
<style>
.advanced-cell {
    background-color: #e84c2250;
}
.advanced-cell::after {
    position: absolute;
    display: block;
    top: -2px;
    right: -2px;
    width: 5px;
    height: calc(100% + 3px);
    content: '';
    background: #e84c22;
}
.advanced-label-row {
    border-bottom: 1px solid #e84c22;
    display: flex;
    font-weight: bold;
}
.advanced-label {
    margin-left: auto;
    background-color: #e84c22;
    padding: 5px 8px;
    color: white;
    margin-right: -2px;
}
</style>
<script>

// A function to hide/show highlight advanced topics in the notebook
var highlighted = false;
function highlight_advanced_topics() {
    $(".advanced-cell").removeClass("advanced-cell");
    $(".advanced-label-row").remove();
    if(highlighted) {
        highlighted = false;
        return;
    }
    var advanced = false;
    $(".jp-Cell.jp-MarkdownCell,.jp-Cell.jp-CodeCell").each(function(){
        if(!advanced) {
            if($(this).find(".advanced-start").length > 0) {
                $(this).before("<div class='advanced-label-row'><span class='advanced-label'>Advanced Topic</span></div>");
                $(this).addClass("advanced-cell");
                advanced = true;
            }        
        } else {
            if($(this).find(".advanced-stop").length > 0) {
                if($(this).find(".advanced-start").length > 0) {
                    $(this).before("<div class='advanced-label-row' style='margin-top: 10px;'><span class='advanced-label'>Advanced Topic</span></div>");
                    $(this).addClass("advanced-cell");
                } else {
                    advanced = false;
                }
            } else {
                $(this).addClass("advanced-cell");
            }
        }
    });
    highlighted = true
}

(function() {
  // Load the script
  const script = document.createElement("script");
  script.src = 'https://ajax.googleapis.com/ajax/libs/jquery/3.6.0/jquery.min.js';
  script.type = 'text/javascript';
  script.addEventListener('load', () => {
    $(document).ready(highlight_advanced_topics);
  });
  document.head.appendChild(script);
})();
</script>
<div class="m-5 p-5"><span class="alert alert-block alert-danger">Advanced topics in notebook are highlighted!</span></div>''')
display(tag)

# Context managers and I/O
There are many instances where our code will have to interact with external resources: just think about reading (writing) data from (to) a file. A common
issue is retaining these resources for longer than necessary, or worse forever. These are sometimes called memory leaks because each resource requires memory and the available memory is reduced each time, for instance, a new file is opened without closing previously used ones.

We would like to isolate each of these *interactions* so that we can be sure that the resources acquired are eventually properly released. Python support us with the concept of *contexts*. At the end of the context, our program is no longer bound to the resource and we want to be guaranteed that the resource is released.

We delegate all the responsibilities of managing resources to their respective context managers, relaxing the need of taking care of their lifecycle and the management of exceptions and corner cases.

## Context managers behaviour
Let's say we want to read/write data from/to a text file. We are aware that something bad could happen when opening a file, so we encapsulate our code in a `try/finally` block. For the sake of this example, we ignore explicitly handling exceptions. A possible approach to manage such a file would be the following one.

In [ ]:
filename = 'path/to/file'
fd = open(filename)
try:
    do_domething(fd)
finally:
    fd.close()

A more Pythonic way to obtain the same result, leveraging the benefits of context managers, is instead the following one.

In [ ]:
filename = 'path/to/file'
with open(filename) as fd:
    do_domething(fd)

The `with` statement enters the context manager. The `open()` function applies the context manager protocol, guaranteeing the file will be automatically closed when the block is finished, even if an exception is thrown.

<span class="advanced-start"></span>
Context managers rely on two magic methods: `__enter__` and `__exit__`. On the first line of the context manager, the `with` statement will call the first method, `__enter__`, and whatever this method returns will be assigned to the variable labeled after `as`. 
After this line is executed, the code enters a new context, where any other Python block can be run. After the last statement on that block is finished, the context will be exited, meaning that the interpreter will call the `__exit__` method of the original context manager object we first invoked.
If there is an exception or error inside the context manager block, the `__exit__` method will still be called, which makes it convenient for safely managing the cleaning up of conditions. 
We can implement our own context managers in order to handle the particular logic we need, but this topic is more suitable for a lesson in the future.

<span class="advanced-stop"></span>
To recap, a context manager is notified of three significant events:
- Entry
- Normal exit without an exception
- Abnormal exit with an exception pending

The context manager will always *detach* our code from external resources. Files will be closed, network connections will be dropped, database transactions will be committed or rolled back, locks will be released and so on and so forth.

# Common patterns for I/O

## Reading data from text files
When you want to work with the information in a text file, the first step is to read the file into memory. 
You can read the entire contents of a file, or you can work through the file one line at a time.

In [3]:
# %load students.txt
Mario Rossi s100200
Giulia Bianchi s100201
Andrea Verdi s100202

In [7]:
# Read the whole file in memory
with open('students.txt') as file:
    students = file.read()
print(students)

Mario Rossi s100200
Giulia Bianchi s100201
Andrea Verdi s100202


In [6]:
# Read the file one line at a time
with open('students.txt') as file:
    for line in file:
        print(line)

Mario Rossi s100200

Giulia Bianchi s100201

Andrea Verdi s100202


We can see that there are extra blank lines after each line. This is because a newline character is at the end of each line in the text file. 
The `print` function, in turn, adds its own newline each time we call it, so we end up with two newlines. We can use `rstrip()` on each line to eliminate the additional ones.

In [12]:
# Read the file one line at a time, cleaning the input.
with open('students.txt') as file:
    for line in file:
        print(line.rstrip())

Mario Rossi s100200
Giulia Bianchi s100201
Andrea Verdi s100202


We can also read a file and concurrently create a list with the lines read, in order to make the contents available once we exit from the context scope.

In [11]:
# Read the file, storing data in a list
with open('students.txt') as file:
    lines = file.readlines()
print(lines) # We can explicitly see the newline character we talked about before

['Mario Rossi s100200\n', 'Giulia Bianchi s100201\n', 'Andrea Verdi s100202']


## Writing data to text files
To write text to a file, we need to call `open()` with a second argument (`w` for write), telling the interpreter what we want to do. We didn't have to specify it in the previous examples because, by defaults, files are opened in read mode. The `write()` function doesn't add newlines to the text we write, we have to include them explicitly.

In [27]:
with open('output_file.txt', 'w') as file:
    file.write("This will be written to file")
    file.write("This will also be written to file\n")
    file.write("This line will be second one.")

In [29]:
# %load output_file.txt
This will be written to fileThis will also be written to file
This line will be second one.

## Reading data from CSV files
A CSV file (Comma Separated Values) is a type of plain text file that structurally stores tabular data. 
Usually, CSV files use a comma to separate each specific data value.

In [31]:
# %load students.csv
Name,Surname,Student ID
Mario,Rossi,s100200
Giulia,Bianchi,s100201
Andrea,Verdi,s100202

We could read a csv file like any other text file, but the `csv` module come in handy for managing such a format. We can read data using a `reader` object.

In [34]:
import csv
with open('students.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            print(f'\t{row[1]} {row[0]} student ID is {row[2]}.')
            line_count += 1
    print(f'Processed {line_count} lines.')

Column names are Name, Surname, Student ID
	Rossi Mario student ID is s100200.
	Bianchi Giulia student ID is s100201.
	Verdi Andrea student ID is s100202.
Processed 4 lines.


## Writing data to CSV files
We can also write to a CSV file using a `writer` object and the `writerow()` method. Such an object tries to terminate each line with universal newlines, and such an option may have to be handled explicitly if the outcome does not match the expected results. 

In [41]:
import csv
with open('students.csv', mode='a', newline='') as csv_file: # Let's append rather than write a new file
    csv_writer = csv.writer(csv_file, delimiter=',') 
    csv_writer.writerow(['Neri', 'Simone', 's100203'])
    csv_writer.writerow(['Grigi', 'Maria', 's100204'])

In [42]:
# %load students.csv
Name,Surname,Student ID
Mario,Rossi,s100200
Giulia,Bianchi,s100201
Andrea,Verdi,s100202
Neri,Simone,s100203
Grigi,Maria,s100204


More I/O examples will be provided as needed during future lessons.